In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

cmap = 'Greens'

# Решающие деревья
Рассмотрим ещё один алгоритм машинного обучения, преимущественно применяемого для классификации. Однако, при должном умении его можно применить и для регресии. Однако делать этого мы, конечно же, не будем.

Решающие деревья появились как попытка формализовать принятие решений человеком. Например, при постановке врачом диагноза. Для постановки диагноза врач задаёт уточняющие вопросы, чтобы ограничить область возможных диагнозов. Прелесть такого подхода в том, что алгоритмы легко интерпретируются человеком.

Сам алгоритм представляет собой дерево, в узлах которого находится некоторый предикат, в результате применения которого к признакам постепенно усекается пространство признаков до тех пор, пока в нём не окажутся объекты одного класса. Постараемся ответить на вопрос о том, как выбирать предикат.

Пример взят [из курса ODS](https://habr.com/company/ods/blog/322534/#derevo-resheniy)

Для старта нам понадобится освежить в памяти понятие энтропии Шеннона. Для системы с N возможными состояниями формула выглядит как:
$$S = -\sum_{i=1}^{N}p_i \log_2{p_i},$$
где $p_i$ - вероятность нахождения системы в $i$-ом состоянии.

Допустим, есть у нас вот такая последовательность шариков из двух цветов - 9 синих и 11 оранжевых.

<img src="img/1.png">

Рассчитаем энтропию такой системы. Для наугад выбранного x вероятность вытащить синий шарик: $p_1 = \frac{9}{20}$, а оранжевый: $p_2 = \frac{11}{20}$. Посчитаем энтропию такой системы:

In [ ]:
p1 = 9 / 20
p2 = 11 / 20

S = -p1 * np.log2(p1) - p2 * np.log2(p2)
print(S)

Теперь разобьём выборку на две по порогу $x \le 12$:

<img src="img/2.png">

А теперь посчитаем энтропию для каждой из выборок:

In [ ]:
p11 = 8 / 13
p12 = 5 / 13

p21 = 1 / 7
p22 = 6 / 7

S1 = -p11 * np.log2(p11) - p12 * np.log2(p12)
S2 = -p21 * np.log2(p21) - p22 * np.log2(p22)

print(S1, S2)
print(np.mean([S1, S2]))

Вот мы и подошли к правилу: 
> Разбив исходный набор данных на две части по некому предикату, можно рассчитать энтропию каждого подмножества, после чего рассчитать среднее значение энтропии — если оно окажется меньшим чем энтропия исходного множества, значит предикат содержит некую обобщающую информацию о данных.

Следовательно, нужно выбирать такие предикаты, которые несут наибольшее падение энтропии. Выберем разбиение, которое даст минимум средней энтропии:

In [ ]:
balls = np.array([0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
                 dtype=bool)
mean = []
for split in range(1, len(balls) - 1):
    p12 = np.sum(balls[:split]) / balls[:split].size
    p11 = 1 - p12

    p22 = np.sum(balls[split:]) / balls[split:].size
    p21 = 1 - p22

    S1 = -p11 * np.log2(p11) - p12 * np.log2(p12)
    S2 = -p21 * np.log2(p21) - p22 * np.log2(p22)

    mean.append(np.mean([S1, S2]))

plt.plot(mean)
_ = plt.grid()

Вот мы и видим, что разбиение по 12 было выбранно неспроста.

## Критерий Джини

Энтропийный критерий - далеко не единственный. Более того, в реализации решающих деревьев библиотеки `sklearn` по умолчанию используется критерий Джини. 

Изначально критерий Джини был статистический показатель степени расслоения общества данной страны или региона по отношению к какому-либо изучаемому признаку. Вычисляется как отношение площади фигуры, образованной кривой Лоренца и кривой равенства, к площади треугольника, образованного кривыми равенства и неравенства. Подробнее об этом можно почитать в [англоязычной вики](https://en.wikipedia.org/wiki/Gini_coefficient), ибо в русскоязычной статья неприлично куцая.

Более интуитивная формулировка: численное значение критерия Джини есть *вероятность неправильной классификации*, если предсказывать классы с вероятностями их появления в этом узле.

В конце тетрадки будет визуализация дерева с количественным значением критерия Джини в каждом узле. Это должно закрепить интуитивное понимания его численного значения.

## Практика

Выполним задачу классификации ирисов Фишера используя решающие деревья. Для начала, вспомним как выглядят данные:

In [ ]:
iris = sns.load_dataset('iris')
X = iris[['petal_width', 'petal_length']].values
y = iris.species.copy()

for i, species in enumerate(iris.species.unique()):
    y.loc[y.eq(species)] = i

plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap, edgecolors='k')
plt.xlabel('Ширина лепестка')
plt.ylabel('Длина лепестка')

Воспользуемся реализацией решающего дерева из библиотеки `sklearn.tree`. Класс называется `DecisionTreeClassifier`.

Воспользуемся парамтерами:
* `criterion='entropy'` - чтобы критерием построения предиката выступало уменьшение энтропии
* `max_depth=1` - глубина дерева, т.е. количество предикатов. Будем увеличивать.
* `random_state=42` - для воспроизводимости результатов

In [ ]:
from sklearn.tree import DecisionTreeClassifier


# Напишем вспомогательную функцию, которая будет возвращать решетку для дальнейшей визуализации.
def get_grid(data):
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    return np.meshgrid(np.arange(x_min, x_max, 0.01),
                       np.arange(y_min, y_max, 0.01))


iris = sns.load_dataset('iris')
X = iris[['petal_width', 'petal_length']].values
y = iris.species.copy()

for i, species in enumerate(iris.species.unique()):
    y.loc[y.eq(species)] = i

# немного кода для отображения разделяющей поверхности
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
xx, yy = get_grid(X)

for i, ax in enumerate(axes.flatten()):
    max_depth = i + 1

    clf_tree = DecisionTreeClassifier(criterion='entropy',
                                      max_depth=max_depth,
                                      random_state=42)
    clf_tree.fit(X, y)

    predicted = clf_tree.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(
        xx.shape)

    ax.pcolormesh(xx, yy, predicted, cmap=cmap, alpha=0.1,
                  shading='gouraud')  # cmap определён в самой первой ячейке
    ax.scatter(X[:, 0],
               X[:, 1],
               c=y,
               cmap=cmap,
               edgecolors='black',
               linewidth=1.5)
    ax.set_title(f'max depth = {max_depth}')

А теперь позволим дереву переобучиться:

In [ ]:
from sklearn.tree import DecisionTreeClassifier


# Напишем вспомогательную функцию, которая будет возвращать решетку для дальнейшей визуализации.
def get_grid(data):
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    return np.meshgrid(np.arange(x_min, x_max, 0.01),
                       np.arange(y_min, y_max, 0.01))


iris = sns.load_dataset('iris')
X = iris[['petal_width', 'petal_length']].values
y = iris.species.copy()

for i, species in enumerate(iris.species.unique()):
    y.loc[y.eq(species)] = i

xx, yy = get_grid(X)

entorpy_tree = DecisionTreeClassifier(criterion='entropy', random_state=42)
gini_tree = DecisionTreeClassifier(criterion='gini', random_state=42)

entorpy_tree.fit(X, y)
gini_tree.fit(X, y)

ctiterion = 'entropy'
if ctiterion == 'entropy':
    classifier = entorpy_tree
elif ctiterion == 'gini':
    classifier = gini_tree

predicted = classifier.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

fig = plt.figure(figsize=(10, 8))
plt.pcolormesh(xx, yy, predicted, cmap=cmap, alpha=0.1, shading='gouraud')
plt.scatter(X[:, 0],
            X[:, 1],
            c=y,
            cmap=cmap,
            edgecolors='black',
            linewidth=1.5)

Что плохого? А давайте отрежем 20% данных для обучения, обучим на них и посмотрим, как поведёт себя дерево:

In [ ]:
from sklearn.tree import DecisionTreeClassifier


# Напишем вспомогательную функцию, которая будет возвращать решетку для дальнейшей визуализации.
def get_grid(data):
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    return np.meshgrid(np.arange(x_min, x_max, 0.01),
                       np.arange(y_min, y_max, 0.01))


iris = sns.load_dataset('iris')
iris = iris.sample(frac=1, replace=False, random_state=42)

X = iris[['petal_width', 'petal_length']].values
y = iris.species.copy()

split = int(y.size * 0.8)

for i, species in enumerate(iris.species.unique()):
    y.loc[y.eq(species)] = i

xx, yy = get_grid(X)

entorpy_tree = DecisionTreeClassifier(criterion='entropy', random_state=42)
gini_tree = DecisionTreeClassifier(criterion='gini', random_state=42)

entorpy_tree.fit(X[:split], y[:split])
gini_tree.fit(X[:split], y[:split])

ctiterion = 'entropy'
if ctiterion == 'entropy':
    classifier = entorpy_tree
elif ctiterion == 'gini':
    classifier = gini_tree

fig = plt.figure(figsize=(10, 8))
predicted = classifier.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
plt.pcolormesh(xx,
               yy,
               predicted,
               cmap=cmap,
               alpha=0.1,
               edgecolors=None,
               shading='gouraud')
plt.scatter(X[:, 0],
            X[:, 1],
            c=y,
            cmap=cmap,
            edgecolors='black',
            linewidth=1.5)

Мы видим, что дерево переобучилось. О том, что с этим делать мы разберёмся дальше

## Визуализация решающих деревьев

Какое главное достоинство решающих деревьев? Их интерпретируемость! Мы всегда можем проследить логику принятия решения. Как? Можно визуализировать все вершины, которые содержат предикаты.

Чтобы построить вершины обученного дерева нужно воспользоваться пакетом `graphviz`.

*N.B. Возможно, на голой анаконде этот код не запустится. Попробуйте поставить модуль* `graphviz` *отдельно. Курите конда преферанс*

In [ ]:
import graphviz
from sklearn import tree

### Вариант с энтропией

In [ ]:
nodes = tree.export_graphviz(entorpy_tree, out_file=None)
graph = graphviz.Source(nodes)
graph

На случай если не выйдет (или лениво) поставить `graphviz`, прикрепляю граф дерева в виде картинки
<img src="img/entropy.png">

### Вариант с критерием Джини

In [ ]:
nodes = tree.export_graphviz(gini_tree, out_file=None)
graph = graphviz.Source(nodes)
graph

И снова в виде пикчи:
<img src="img/gini.png">